# Import packages

In [ ]:
import panel as pn
import pandas as pd
import holoviews as hv
import geoviews as gv
from holoviews.plotting.links import Link
from holoviews.plotting.bokeh import LinkCallback
import geoviews.tile_sources as gts
from bokeh.models import HoverTool

hv.extension('bokeh', width=100)
pn.extension()
gv.extension('bokeh')

# Choose language for HTML document!

In [ ]:
language = 'en'  # 'en' or 'de'

# Load data and photos

In [ ]:
from glacier_data import glaciers

In [ ]:
# load some useful extra information for all glacier world wide
extra_data = ['CenLon', 'CenLat', 'Lmax', 'TerminusType']
df_extra_info = pd.read_hdf('./data/rgi62_era5_itmix_df.h5', 'df')
df_extra_info = df_extra_info[extra_data].copy()

# define empty final dataframe with all columns
img_data = ['rgi_id', 'name', 'type', 'img', 'img_pretext',
            'img_cite', 'img_license', 'img_license_link',
            'img_width', 'img_height', 'img_source']
df = pd.DataFrame(columns = extra_data + img_data)

# loop through all glaciers and fill final dataframe
for glacier in glaciers:
    # test all necessary keys are defined
    if not all(k in glacier for k in img_data):
        raise ValueError(f'keys {[k for k in img_data if k not in glacier]} '
                         f'not defined for {glacier}!')

    # get extra glacier info
    glacier_info = df_extra_info.loc[df_extra_info.index == glacier['rgi_id']].to_dict(orient='records')[0]

    # combine image informations with extra info and add to final dataframe
    glacier.update(glacier_info)
    glacier_df = pd.DataFrame([glacier])
    df = pd.concat([df, glacier_df])

# need to save coordinates in extra field for hover
# (original CenLon and CenLat are reprojected during plotting)
df.loc[:, 'LonDeg'] = df['CenLon']
df.loc[:, 'LatDeg'] = df['CenLat']

# set rgi_id as index
df = df.set_index('rgi_id')

# build containers for img links and license links
links_img_src = '<div>'
links_to_license_elements = []
for i, gl in df.iterrows():
    links_img_src += f'<a href="{gl["img_source"]}">{gl["name"]}</a><br>'
    
    if gl["img_license"] != '' and gl["img_license_link"] != '':
        tmp_license = (f'<a href="{gl["img_license_link"]}">'
                       f'{gl["img_license"].replace("(", "").replace(")", "")}</a><br>')
        if tmp_license not in links_to_license_elements:
            links_to_license_elements.append(tmp_license)

links_img_src += '</div>'
links_to_license = '<div>' + ''.join(links_to_license_elements) + '</div>'

# Define texts in according language

In [ ]:
# import all texts in the corresponding language from here:
from international import trans

In [ ]:
title                 = '<div style="font-size:38px; color: #326a86; font-weight: bold;" >{}</div>'.format(trans['Title_hp'][language])
instruction           = '<div style="font-size:15px; color: #326a86; text-align: center;" >{}</div>'.format(trans['instruction'][language])
link_to_license_intro = '<div>{}</div>'.format(trans['link_to_license_intro'][language])
links_img_src_intro   = '<div>{}</div>'.format(trans['links_img_src_intro'][language])

# translate "terminating" entry:
df.loc[df['TerminusType'] == 'Land-terminating', 'TerminusType']   = trans['TerminusType_land'][language]
df.loc[df['TerminusType'] == 'Marine-terminating', 'TerminusType'] = trans['TerminusType_marine'][language]

# translate "type" entry:
df.loc[df['type'] == 'Pure ice glacier', 'type']       = trans['type_pure_ice'][language]
df.loc[df['type'] == 'Calving glacier', 'type']        = trans['type_calving'][language]
df.loc[df['type'] == 'Debris covered glacier', 'type'] = trans['type_debris'][language]
df.loc[df['type'] == 'Ice cap', 'type']                = trans['type_ice_cap'][language]

# translate "img_pretext":
df.loc[df['img_pretext'] == 'Photo courtesy', 'img_pretext'] = trans['img_pretext'][language]

# define colors for dots of different glacier types
glacier_type_colors = {trans['type_pure_ice'][language]: '#56B4E9',
                       trans['type_calving'][language]: '#F0E442',
                       trans['type_debris'][language]: '#009E73',
                       trans['type_ice_cap'][language]: '#D55E00'}

# Design for map with hover and webpage

In [ ]:
# background map
background_map = gts.tile_sources['CartoLight']

# define hover with image and values to display
TOOLTIPS =  """
    <div>
        <div>
            <img
                src="@img" 
                height="@img_height" 
                alt="image loading..." 
                width="@img_width"
                style=" margin: 2px 2px 0px 2px;"
                border="2">
            </img>
        </div>
        <div>
                <span style="font-size: 17px; font-weight: bold;">@name                </span> <br>
                <span style="font-size: 15px;                   ">@type                </span> <br>
                <span style="font-size: 15px;                   ">{}:              </span>
                <span style="font-size: 15px;                   ">@Lmax                </span>
                <span style="font-size: 15px;                   ">m                    </span> <br>
                <span style="font-size: 15px;                   ">@TerminusType        </span> <br>
                <span style="font-size: 15px;                   ">{}:            </span>
                <span style="font-size: 13px; color: #696;      ">(@LatDeg, @LonDeg)   </span> <br>
                <span style="font-size: 9px;                    ">@img_pretext @img_cite @img_license </span>
        </div>
    </div>
            """.format(trans['tooltip_length'][language], trans['tooltip_loc'][language])
hover = HoverTool(tooltips=TOOLTIPS)

# glacier points, include all needed data (everything from hover info to text description) in vdims
glacier_points = gv.Points(df,
                           kdims=['CenLon', 'CenLat'],
                           vdims=['type', 'name', 'Lmax', 'TerminusType', 'img_pretext', 'img_cite',
                                  'img_license', 'img', 'img_height', 'img_width', 'LatDeg', 'LonDeg'
                                 ]
                          ).opts(default_tools=['tap', 'wheel_zoom', 'reset', 'pan', hover],
                                 size=10,
                                 line_color='black',
                                 fill_alpha=0.9,
                                 fill_color='type',
                                 cmap=glacier_type_colors,
                                 xaxis=None,
                                 yaxis=None,
                                )

# put together the total figure
total_map = (glacier_points * background_map).opts(width=1000,
                                                   height=650,
                                                   legend_opts={'click_policy': 'none'},
                                                  )

# texts for different tabs
glacier_definition_text = hv.Div(f"<h2>{trans['Glacier_title'][language]}</h2>"
                                 f"{trans['Glacier_definition'][language]}<br><br>"
                                 f"<i>{trans['Glacier_text_source'][language]}</i>"
                                ).opts(sizing_mode='stretch_both')

pure_ice_text = hv.Div(f"<h2>{trans['type_pure_ice'][language]}</h2>"
                       f"{trans['pure_ice_text'][language]}<br><br>"
                       f"<i>{trans['pure_ice_source'][language]}</i>"
                      ).opts(sizing_mode='stretch_both')

calving_text = hv.Div(f"<h2>{trans['type_calving'][language]}</h2>"
                      f"{trans['calving_text'][language]}<br><br>"
                      f"<i>{trans['calving_source'][language]}</i>"
                     ).opts(sizing_mode='stretch_both')

debris_text = hv.Div(f"<h2>{trans['type_debris'][language]}</h2>"
                     f"{trans['debris_text'][language]}<br><br>"
                     f"<i>{trans['debris_source'][language]}</i>"
                    ).opts(sizing_mode='stretch_both')

ice_cap_text = hv.Div(f"<h2>{trans['type_ice_cap'][language]}</h2>"
                      f"{trans['ice_cap_text'][language]}<br><br>"
                      f"<i>{trans['ice_cap_source'][language]}</i>"
                     ).opts(sizing_mode='stretch_both')

# put together tab with texts
tabs = pn.Tabs((trans['tab_title_glacier_definition'][language],
                glacier_definition_text),
               (trans['tab_title_pure_ice'][language],
                pure_ice_text),
               (trans['tab_title_calving'][language],
                calving_text),
               (trans['tab_title_debris'][language],
                debris_text),
               (trans['tab_title_ice_cap'][language],
                ice_cap_text),
               width=400)

# this Div's is a hacky workaround to link the hv selection to a pn object
linking_div_hv = hv.Div(' ')
linking_div_pn = pn.panel(linking_div_hv, visible=False)

# first we link the plot to hv Div
class TextLinkhv(Link):
    _requires_target = True

class TextCallbackhv(LinkCallback):
    
    source_model = 'selected'
    source_handles = ['cds']
    on_source_changes = ['indices']

    target_model = 'plot'
    
    source_code = """
        var inds = source_selected.indices

        if (inds.length == 0) {
            target_plot.text = ' '
        } else {
            target_plot.text = source_cds.data['type'][inds[0]]
        }
    """

TextLinkhv.register_callback('bokeh', TextCallbackhv)
TextLinkhv(glacier_points, linking_div_hv)

# Now we link the pn Div to the tabs
code_pn_link = f"""
    if (source.text == ' ')
        target.active = 0
    if (source.text == '{trans['type_pure_ice'][language]}')
        target.active = 1
    if (source.text == '{trans['type_calving'][language]}')
        target.active = 2
    if (source.text == '{trans['type_debris'][language]}')
        target.active = 3
    if (source.text == '{trans['type_ice_cap'][language]}')
        target.active = 4
"""

linking_div_pn.jslink(tabs, code={'text': code_pn_link});

# Logos

In [ ]:
oggm_logo = ('<p style="margin-top: 0px;margin-bottom: 0px;"><a href="http://edu.oggm.org">'
             '<img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220>'
             '</a></p>')
hv_logo = ('<p style="margin-top: 0px;margin-bottom: 0px;"><a href="https://holoviz.org">'
           '<img src="https://holoviz.org/assets/holoviz-logo-stacked.svg" width=80>'
           '</a></p>')
fk_logo = ('<p style="margin-top: 0px;margin-bottom: 0px;"><a href="https://www.uibk.ac.at/foerderkreis1669/">'
           '<img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/uibk_logo_narrow.png" width=140>'
           '</a></p>')

# Design application

In [ ]:
# build header
header_height = 98
oggm_logo_pn = pn.Column(pn.Spacer(sizing_mode='stretch_height',
                                   margin=(0,0)),
                         pn.panel(oggm_logo,
                                  margin=(0, 0),
                                  height=70),
                        pn.Spacer(sizing_mode='stretch_height',
                                   margin=(0,0)),
                        height=header_height)

title_instruction_pn = pn.Column(
    pn.Row(pn.Spacer(sizing_mode='stretch_width', margin=(0, 0)),
           pn.panel(title, margin=(0, 0),
                   ),
           pn.Spacer(sizing_mode='stretch_width', margin=(0, 0)),
           sizing_mode='stretch_width'),
    pn.Row(pn.Spacer(sizing_mode='stretch_width', margin=(0, 0)),
           pn.panel(instruction, margin=(0, 0),
                    width=800,),
           pn.Spacer(sizing_mode='stretch_width', margin=(0, 0)),
           margin=(0, 0),
           sizing_mode='stretch_width'),
    margin=(0, 0),
    sizing_mode='stretch_width')

logos_pn = pn.Column(
    pn.Spacer(sizing_mode='stretch_height',
                                   margin=(0,0)),
    pn.Row(pn.Pane(fk_logo,
                   align='center',
                   height=85,
                   margin=(0, 0)),
           pn.Spacer(width=50, margin=(0, 0)),
           pn.Pane(hv_logo,
                   align='center',
                   height=85,
                   margin=(0, 0))
          ),
    pn.Spacer(sizing_mode='stretch_height',
                                   margin=(0,0)))
header = pn.Row(oggm_logo_pn, 
                pn.Spacer(width=10,
                          height=10,
                          margin=(0, 0)), 
                title_instruction_pn,
                pn.Spacer(width=10,
                          height=10,
                          margin=(0, 0)),
                logos_pn,
                pn.Spacer(width=30,
                          height=10,
                          margin=(0, 0)),
                margin=(0, 0),
                sizing_mode='stretch_width'
                )

# links for image source and licenses
links = pn.Row(pn.Column(links_img_src_intro,
                         links_img_src,),
               pn.Spacer(width=10),
               pn.Column(link_to_license_intro,
                         links_to_license,
                         width=350),
              )

# Put plot and text together
app = pn.Column(header,
                pn.Spacer(height=2),
                pn.Row(total_map,
                       pn.Spacer(width=2),
                       tabs,
                       sizing_mode='stretch_both'),
                pn.Spacer(height=10),
                links,
                linking_div_pn,
                margin=(0, 0),
                sizing_mode='stretch_both')

#app.show()

# Save application as HTML

In [ ]:
# save as a stand-alone HTML
file_name = f'gallery-app_{language}.html'
app.save(file_name, 'Glacier Gallery')